# Limpieza dato de estación

Este script realiza la lectura de los datos de eventos que está en formato TXT, hace un pre-procesamiento de los mismos e guarda dentro de una planilla CSV para que sean usados posteriormente dentro de los análisis.

Para una misma estácion, todos los datos serna juntados para criar una única tabela


In [1]:
import pandas as pd
import numpy as np
import os
import re
from functools import reduce
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
PATH_DRIVE = 'drive/MyDrive/Seminario-3-master'
PATH_DADOS = '/preprocess/dados/'
NOMBRE_ESTACION = 'chucuri'
PATH_TIPO_DADO = '/hora/'
FILE_DATOS_PROCESADOS = NOMBRE_ESTACION.lower() + '_procesado.csv'

PATH_FINAL_DADOS = PATH_DRIVE + PATH_DADOS + NOMBRE_ESTACION + PATH_TIPO_DADO

In [15]:
fname = os.listdir(PATH_FINAL_DADOS)

list_df = []

for file in fname:
    print(file)
    name_column_data = re.search(r"^.*_(.*)\.txt", file).group(1)
    with open(f'{PATH_FINAL_DADOS}{file}') as f:
        next(f); next(f) # Saltando dos lineas
        list_data = []
        for line in f:
            dict_data = {}
            if line:
                line_split = line.strip().split()
                dict_data['FECHA'] = ''.join(line_split[0:2])
                dict_data[name_column_data] = line_split[2]
                list_data.append(dict_data)
    
    df = pd.DataFrame(list_data)
    list_df.append(df)
                

CHUCURI_24055070_HUM.txt
CHUCURI_24055070_TEMP.txt
CHUCURI_24055070_PRESION.txt


In [16]:
# Reduciendo dataframes en uno solo
df = reduce(lambda df1,df2: pd.merge(df1,df2,on='FECHA', how='outer'), list_df)

# Formatando data
df['FECHA'] = pd.to_datetime(df['FECHA'], format='%Y%m%d%H:%M:%S').dt.tz_localize('America/Bogota')

In [19]:
df

,FECHA,HUM,TEMP,PRESION
0,2011-10-24 01:00:00-05:00,NaN,NaN,NaN
1,2011-10-24 02:00:00-05:00,NaN,NaN,NaN
2,2011-10-24 03:00:00-05:00,NaN,NaN,NaN
3,2011-10-24 04:00:00-05:00,NaN,NaN,NaN
4,2011-10-24 05:00:00-05:00,NaN,NaN,NaN
...,...,...,...,...
50278,2017-12-25 04:00:00-05:00,NaN,NaN,NaN
50279,2017-12-25 05:00:00-05:00,NaN,NaN,NaN
50280,2017-12-25 06:00:00-05:00,NaN,NaN,NaN
50281,2017-12-25 07:00:00-05:00,NaN,NaN,NaN


In [18]:
columns = [col for col in df.columns if col != 'FECHA']

for col in columns:
    # Substituyendo valores no encontrados por np.nan
    df[col] = df[col].replace(['---[10]', '---[2]', '---[5]', '---[15]', '---'], np.NaN)
    # Transformando en formato numérico
    df[col] = pd.to_numeric(df[col], downcast="float")

In [20]:
df.to_csv(PATH_FINAL_DADOS + '../' + FILE_DATOS_PROCESADOS, header=True, index=False, sep = ';')